<a href="https://colab.research.google.com/github/TheAmirHK/simplexAl/blob/main/Simplex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
import numpy as np

def simplex(c, A, b):
    c = np.array(c, dtype=float)
    A = np.array(A, dtype=float)
    b = np.array(b, dtype=float)

    num_vars = len(c)
    num_constraints = len(A)

    tableau = np.hstack([A, np.eye(num_constraints), b.reshape(-1, 1)])
    c_row = np.hstack([-c, np.zeros(num_constraints + 1)])
    tableau = np.vstack([tableau, c_row])

    var_labels = [f"x{i+1}" for i in range(num_vars)] + [f"s{i+1}" for i in range(num_constraints)]

    step = 0
    print_tableau(tableau, step, var_labels)

    while True:
        last_row = tableau[-1, :-1]
        if np.all(last_row >= 0):
            break

        entering_col = np.argmin(last_row)

        ratios = tableau[:-1, -1] / tableau[:-1, entering_col]
        ratios[ratios <= 0] = np.inf
        leaving_row = np.argmin(ratios)

        pivot_element = tableau[leaving_row, entering_col]
        tableau[leaving_row] /= pivot_element

        for i in range(len(tableau)):
            if i != leaving_row:
                tableau[i] -= tableau[i, entering_col] * tableau[leaving_row]

        step += 1
        print_tableau(tableau, step, var_labels)

    solution = np.zeros(num_vars)
    for i in range(num_vars):
        column = tableau[:-1, i]
        if np.sum(column == 1) == 1 and np.sum(column) == 1:
            row_index = np.where(column == 1)[0][0]
            solution[i] = tableau[row_index, -1]

    optimal_value = tableau[-1, -1]

    return optimal_value, solution

def print_tableau(tableau, step, var_labels):

    print(f"\nStep {step}:")
    print("    " + "  ".join(var_labels + ["RHS"]))
    for row in tableau:
        print("  ".join(f"{val:6.2f}" for val in row))
    print("-" * 60)

In [31]:
# example

c = [5, 4, -3]  # maximize 5x1 + 4x2 - 3x3
A = [[20, 3, 1],  # 20x1 + 3x2 + x3 ≤ 50
     [4, -1, 2],  # 4x1 - x2 + 2x3 ≤ 110
     [3, 4, 2]]  # 3x1 + 4x2 + 2x3 ≤ 8
b = [50, 110, 8]

optimal_value, solution = simplex(c, A, b)


Step 0:
    x1  x2  x3  s1  s2  s3  RHS
 20.00    3.00    1.00    1.00    0.00    0.00   50.00
  4.00   -1.00    2.00    0.00    1.00    0.00  110.00
  3.00    4.00    2.00    0.00    0.00    1.00    8.00
 -5.00   -4.00    3.00    0.00    0.00    0.00    0.00
------------------------------------------------------------

Step 1:
    x1  x2  x3  s1  s2  s3  RHS
  1.00    0.15    0.05    0.05    0.00    0.00    2.50
  0.00   -1.60    1.80   -0.20    1.00    0.00  100.00
  0.00    3.55    1.85   -0.15    0.00    1.00    0.50
  0.00   -3.25    3.25    0.25    0.00    0.00   12.50
------------------------------------------------------------

Step 2:
    x1  x2  x3  s1  s2  s3  RHS
  1.00    0.00   -0.03    0.06    0.00   -0.04    2.48
  0.00    0.00    2.63   -0.27    1.00    0.45  100.23
  0.00    1.00    0.52   -0.04    0.00    0.28    0.14
  0.00    0.00    4.94    0.11    0.00    0.92   12.96
------------------------------------------------------------


In [32]:
print("\nOptimal value:", optimal_value)
print("Optimal solution:", solution)


Optimal value: 12.95774647887324
Optimal solution: [2.47887324 0.14084507 0.        ]
